In [1]:
import pandas as pd
import os

pd.set_option('display.max_columns', 500)
data_path = '../../data/modulo2/examen/'
os.listdir(data_path)

['churn-bigml-80 - churn-bigml-80.csv', 'DataExam1']

In [2]:
filename = 'churn-bigml-80 - churn-bigml-80.csv'
df = pd.read_csv(data_path+filename)

In [3]:
df.shape

(2666, 20)

In [4]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
df['Churn'].value_counts(1, dropna=False)

False    0.854464
True     0.145536
Name: Churn, dtype: float64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
predictors = [x for x in df.columns if 'Total' in x] + ['Number vmail messages', 'Customer service calls']
tar = 'Churn'
X = df[predictors].copy()
y = df[tar].copy()

Xt, Xv, yt, yv = train_test_split(X, y, test_size=.2)

# Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble._forest:

class RandomForestClassifier(ForestClassifier)
 |  RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and uses averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  P

In [10]:
rf = RandomForestClassifier(n_estimators=5, max_depth=3, verbose=2, n_jobs=-1)
rf.fit(Xt, yt)

building tree 1 of 5building tree 2 of 5

building tree 3 of 5
building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


RandomForestClassifier(max_depth=3, n_estimators=5, n_jobs=-1, verbose=2)

In [11]:
print('Entrenamiento: \n')
predictions = rf.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = rf.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 

              precision    recall  f1-score   support

       False       0.87      1.00      0.93      1809
        True       0.88      0.15      0.26       323

    accuracy                           0.87      2132
   macro avg       0.87      0.58      0.60      2132
weighted avg       0.87      0.87      0.83      2132

Matriz: 

[[1802    7]
 [ 273   50]]
Validación: 

              precision    recall  f1-score   support

       False       0.89      1.00      0.94       469
        True       0.78      0.11      0.19        65

    accuracy                           0.89       534
   macro avg       0.83      0.55      0.56       534
weighted avg       0.88      0.89      0.85       534

Matriz: 

[[467   2]
 [ 58   7]]


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


# Práctica revisar qué modelo, podría cubrir el GAP que tiene el Random Forest, para el recall, pues no nos está ayudando a captar muchos de los casos de fuga del cliente.

# Práctica 5

In [64]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [63]:
# help(LogisticRegressionCV)
# help(DecisionTreeClassifier)
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [79]:
# lr = LogisticRegressionCV(max_iter=1000, solver='saga')
dt = DecisionTreeClassifier(max_depth=7)
# nn = MLPClassifier()
# bayes = GaussianNB()
"""
nn = MLPClassifier(max_iter=300, verbose=2, activation='logistic', solver='sgd',
                   hidden_layer_sizes=(200,500, 100, 400),
                   learning_rate='adaptive', shuffle=True)
"""

"\nnn = MLPClassifier(max_iter=300, verbose=2, activation='logistic', solver='sgd',\n                   hidden_layer_sizes=(200,500, 100, 400),\n                   learning_rate='adaptive', shuffle=True)\n"

In [80]:
#lr.fit(Xt, yt)
dt.fit(Xt, yt)
# bayes.fit(Xt, yt)
# nn.fit(Xt, yt)

DecisionTreeClassifier(max_depth=7)

In [81]:
print('Pico Lara Alberto Isaac, mi modelo fue una Árbol de decisión de profundidad 7')
print('Entrenamiento: \n')
predictions = dt.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = dt.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Pico Lara Alberto Isaac, mi modelo fue una Árbol de decisión de profundidad 7
Entrenamiento: 

              precision    recall  f1-score   support

       False       0.94      1.00      0.97      1809
        True       0.99      0.64      0.78       323

    accuracy                           0.94      2132
   macro avg       0.96      0.82      0.87      2132
weighted avg       0.95      0.94      0.94      2132

Matriz: 

[[1806    3]
 [ 115  208]]
Validación: 

              precision    recall  f1-score   support

       False       0.93      0.97      0.95       469
        True       0.68      0.46      0.55        65

    accuracy                           0.91       534
   macro avg       0.81      0.72      0.75       534
weighted avg       0.90      0.91      0.90       534

Matriz: 

[[455  14]
 [ 35  30]]


# Gradient Boosting

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
help(GradientBoostingClassifier)

Help on class GradientBoostingClassifier in module sklearn.ensemble._gb:

class GradientBoostingClassifier(sklearn.base.ClassifierMixin, BaseGradientBoosting)
 |  GradientBoostingClassifier(*, loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for classification.
 |  
 |  GB builds an additive model in a
 |  forward stage-wise fashion; it allows for the optimization of
 |  arbitrary differentiable loss functions. In each stage ``n_classes_``
 |  regression trees are fit on the negative gradient of the
 |  binomial or multinomial deviance loss function. Binary classification
 |  is a special case where only a sin

In [14]:
gb = GradientBoostingClassifier(n_estimators=90, max_depth=3, loss='log_loss', verbose=2, )
gb.fit(Xt, yt)

ValueError: Loss 'log_loss' not supported. 

In [15]:
print('Entrenamiento: \n')
predictions = gb.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = gb.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 



AttributeError: 'GradientBoostingClassifier' object has no attribute 'estimators_'

# XGBoost

In [16]:
from xgboost import XGBClassifier
help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective='binary:logistic', use_label_encoder=True, **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |      use_label_encoder : bool
 |          (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new code,
 |          we recommend that you set this parameter to False.
 |  
 |      max_depth : int
 |          Maximum tree depth for base learners.
 |      learning_rate : float
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : int
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |      objective : string or callable
 |          Specify the learning task and the corresponding learning objective or
 |          a custom objective func

In [17]:
xgb = XGBClassifier(n_estimators=30, n_jobs=-1, verbosity=2, booster='dart')
xgb.fit(Xt, yt)

[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 0 pruned nodes, max_depth=6
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:731: drop 0 trees, weight = 1
[15:54:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=6
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:731: drop 0 trees, weight = 1
[15:54:54] INFO: /home/conda/feedstock_root/build_a

/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:731: drop 0 trees, weight = 1
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:731: drop 0 trees, weight = 1
[15:54:54] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=6
[15:54:55] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/gbm/gbtree.cc:731: drop 0 trees, weight = 1
[15:54:55] INFO: /home/conda/feedstock_root/build_artif

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=30, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=2)

In [18]:
print('Entrenamiento: \n')
predictions = xgb.predict(Xt)
print(classification_report(yt, predictions))
print('Matriz: \n')
print(confusion_matrix(yt, predictions))
print('Validación: \n')
predictions = xgb.predict(Xv)
print(classification_report(yv, predictions))
print('Matriz: \n')
print(confusion_matrix(yv, predictions))

Entrenamiento: 

              precision    recall  f1-score   support

       False       0.97      1.00      0.99      1809
        True       1.00      0.85      0.92       323

    accuracy                           0.98      2132
   macro avg       0.99      0.93      0.95      2132
weighted avg       0.98      0.98      0.98      2132

Matriz: 

[[1809    0]
 [  48  275]]
Validación: 

              precision    recall  f1-score   support

       False       0.93      1.00      0.96       469
        True       0.94      0.48      0.63        65

    accuracy                           0.93       534
   macro avg       0.94      0.74      0.80       534
weighted avg       0.93      0.93      0.92       534

Matriz: 

[[467   2]
 [ 34  31]]
